In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd


In [2]:
def sort_data(text):
        f = open(text, "r")
        lines = f.readlines()
        f.close()

        grid_row, grid_col, n_drones, max_turns, max_payload = [int(lines[0].split()[i]) for i in range(5)]
        n_prod_types = int(lines[1])
        weight_prod_types = lines[2].split()
        weight_prod_types = [int(i) for i in weight_prod_types]         #making elements int
        n_wrhs = int(lines[3])

        wrhs_info = []
        i=4
        while(i<n_wrhs*2+4):
            wrhs_loc, num_itms_per_prodtype = lines[i].split(), lines[i+1].split()
            wrhs_loc = [int(i) for i in wrhs_loc]
            num_itms_per_prodtype = [int(i) for i in num_itms_per_prodtype]
            wrhs_info.append([wrhs_loc, num_itms_per_prodtype])
            i = i+2

        n_orders = int(lines[24])
        i=25
        order_info = []
        while(i<n_orders*3+25):
            order_loc, n_order_items, prod_type_of_prod_item =  lines[i].split(), lines[i+1].split(), lines[i+2].split()
            order_loc = [int(i) for i in order_loc]
            n_order_items = [int(i) for i in n_order_items]
            prod_type_of_prod_item = [int(i) for i in prod_type_of_prod_item]
            order_info.append([order_loc, n_order_items, prod_type_of_prod_item])
            i = i+3
        return grid_row, grid_col, n_drones, max_turns, max_payload, n_prod_types, weight_prod_types, n_wrhs, wrhs_info, n_orders, order_info

In [3]:
data = sort_data("busy_day.in")
grid_row, grid_col, n_drones, max_turns, max_payload, n_prod_types, weight_prod_types, n_wrhs, wrhs_info, n_orders, order_info = [data[i] for i in range(11)]

#-- Geograhical plot of warehouses and orders
wrhs_loc = [wrhs_info[i][0] for i in range(len(wrhs_info))]
order_loc = [order_info[i][0] for i in range(len(order_info))]
wrhs_num = list(range(n_wrhs))

#-- Number of items in each warehouse
wrhs_n_items = [sum(i[1]) for i in wrhs_info]

#-- Dataframe for the Orders [Row Location, Col Location, N of items per order]
x_order_loc = [order_info[i][0][0] for i in range(len(order_info))]
y_order_loc = [order_info[i][0][1] for i in range(len(order_info))]
n_items_per_order = [order_info[i][1][0] for i in range(len(order_info))]
df1_orders = pd.DataFrame(list(zip(x_order_loc,y_order_loc,n_items_per_order)), columns = ["X", "Y", "N of Items"])


In [9]:
class Dataframes:

    def __init__(self):
        data = sort_data("busy_day.in")
        self.grid_row, \
        self.grid_col, \
        self.n_drones, \
        self.max_turns, \
        self.max_payload, \
        self.n_prod_types, \
        self.weight_prod_types, \
        self.n_wrhs, \
        self.wrhs_info, \
        self.n_orders, \
        self.order_info = [data[i] for i in range(11)]

    def get_df_orders(self):
        x_order_loc = [self.order_info[i][0][0] for i in range(len(self.order_info))]
        y_order_loc = [self.order_info[i][0][1] for i in range(len(self.order_info))]
        n_items_per_order = [self.order_info[i][1][0] for i in range(len(self.order_info))]
        df_orders = pd.DataFrame(list(zip(x_order_loc, y_order_loc, n_items_per_order)),
                                 columns=["X", "Y", "N of Items"])
        return df_orders


In [26]:
def dist(a, b):
    if isinstance(a,np.ndarray) or  isinstance(b,np.ndarray):
        return np.sqrt(((a-b)**2).sum(1))
    return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

class Drone():  # inherit #product #warehouse #order (#utility)

    def __init__(self, num):
        self.num = num
        self.products_type = []
        self.products_weight = np.array(weight_prod_types)[self.products_type].sum()
        self.products_quantity = []
        self.pld_mass = 0
        self.cur_pos = [0, 0]
        self.turns = 0  # ?
        self.actions = []  # ?
        self.busy = False

        # Df for use throughout
        self.Data = Dataframes()
    
    def __repr__(self):
        return '(num: ' + str(self.num) + ', ' + 'payload_weight: ' + str(sum(self.products_type)) + ')' 
        
    def load(self, prod_type, qnty):
        self.products_type.append(prod_type)
        self.products_quantity.append(qnty)
        # pactions.append([0 L 1 2 3 ]) ?order_number
        self.turns += 1

    def unload(self, prod_type, qnty):  # warehouse
        self.products_type.remove(prod_type)
        self.products_quantity.remove(qnty)
        # actions.append([0 U 1 2 3 ])?order_number
        self.turns += 1

    def update_pld_mass(self):
        tot_weight = 0
        for i in range(len(self.products_type)):
            unit_weight = self.Data.weight_prod_types[self.products_type[i]]
            tot_weight += unit_weight * self.products_quantity[i]
        self.pld_mass = tot_weight

    def deliver(self, prod_type, qnty):  # to the order
        self.products_type.remove(prod_type)
        self.products_quantity.remove(qnty)
        # actions.append([0 D 1 2 3 ])?order_number
        self.turns += 1

    def wait(self, n_turns):
        self.turns += n_turns

    def get_cur_pos(self):
        return self.cur_pos

    def update_cur_pos(self, new_pos):
        self.cur_pos = new_pos  # define in utility class
        
    def find_nearest_wh(self, warehouses):
        o = np.array([orders[x].position for x in orders], dtype = np.float64)
        c = np.array([orders[x].completed for x in orders])
        if c.sum() == 1250:
            return 'All orders are completed'
        o[c] = np.inf
        d = dist(self.cur_pos, o)
#         w = dist(self.cur_pos, o.warehouses)
        return orders[np.argmin(d)]        
        
    def find_nearest_order(self, orders):
        o = np.array([orders[x].position for x in orders], dtype = np.float64)
        c = np.array([orders[x].completed for x in orders])
        if c.sum() == 1250:
            return 'All orders are completed'
        o[c] = np.inf
        d = dist(self.cur_pos, o)
#         w = dist(self.cur_pos, o.warehouses)
        return orders[np.argmin(d)]

#     def find_best_path(self, order):
#         pass
    
    
#     def complete_order(self, order):
#         products = 
        
# d_0 = Drone(0)
# d_0.update_cur_pos([0,1])
# d_0.find_nearest_order(orders)
# dist([0,1], np.array([orders[o].position for o in orders]))

SyntaxError: invalid syntax (<ipython-input-26-50d043ae9d7a>, line 58)

In [35]:
w.products_quantity

array([ 5,  1,  2,  4,  8, 11,  5,  4,  7, 46,  1,  6,  3,  9,  3,  3,  2,
        3,  1,  4,  3,  2,  1,  3,  2,  2,  9,  2,  5,  1,  4, 13,  1,  5,
       16, 50,  3,  4,  1,  5,  2,  2,  4,  3,  2, 23,  2, 10,  7, 11,  1,
        8,  9,  1,  2,  6,  6,  2,  6,  3,  1,  3,  5,  3,  2,  4,  3,  9,
        2,  2, 10,  4,  4,  2, 15,  1,  4,  1,  5,  6,  5,  2,  6, 67,  8,
        5,  6, 59,  3,  8,  3,  1,  1,  1,  4, 29,  3,  6, 38,  7,  1,  1,
        4,  2,  1,  7,  5,  7,  1,  1,  1,  6,  1,  3,  8,  4,  4,  7,  7,
        9,  7,  6,  4,  4,  1,  3,  2,  8,  8,  3,  5, 10,  6,  1,  2, 46,
       11,  4,  1,  1,  4,  1,  2,  3,  1,  4, 11,  8,  5, 46,  3, 54, 40,
        7,  3,  2, 11,  3,  2,  2,  8,  2,  1,  5,  6, 57,  8, 12,  7,  6,
        2,  4,  1,  2,  4,  1,  2,  4,  2,  4,  2,  2,  1,  3,  1,  3,  2,
        3,  5,  5,  5,  3,  8,  4, 10,  2, 36,  1,  5,  2,  5, 15,  5,  5,
        4,  1,  3,  2,  2, 11,  6,  7])

In [82]:
w = warehouses[0]
order = orders[0]
w.products_types[pd.Series(w.products_types).isin(order.products_type)]

array([ 42, 280, 299], dtype=int64)

In [83]:
order.products_quantity[w.products_types[pd.Series(w.products_types).isin(order.products_type)]]

42     1
280    1
299    1
dtype: int64

In [80]:
a = np.array([8,9,0])
b = np.array([1,2,3])
np.array([a,b]).min(0)

array([1, 2, 0])

In [ ]:
    def compute_routes(self, routes, order):#route is a wharehouse or list of wharehouses 
    r = {}
    for route in routes:
        if isinstance(route, int):
            r[route] = dist(self.get_cur_pos, route.pos) + dist(route.pos, order)
        if isinstance(route, list):
            w = route[-1]
            r[tuple(route)] =  sum([dist(self.get_cur_pos, wh.pos) for wh in route]) + dist(w.pos, order)
    return min(r, key = r.get) 

# #     for each drone find the order that minimizes the distance to travel
routes = [9,
          [1,8],
         [2,3,4,7]]
         [100+1, 100+2]
min[drone.curpos,  [9 + dist(9, order), [1,8] + dist(8, order), [8,1] + dist(1, order)]]

    def find_min_dist(self, order):
        routes = []
    #     compute all the distances from the warehouse, sort them by distance from current position
        dist_wh = pd.DataFrame.from_dict(dict(zip(range(n_wrhs), dist(self.get_cur_pos, wh.pos))), orient = 'index').sort_values(ascending = False)
    #     check all the routes for the order (check availability in wharehouse and return the min of all the distances)
        
        for x in warehouses:
            w = wharehouses[x]
            if np.all(pd.Series(order.products_type).isin(w.products_types)):
                if order.products_quantity < w.products_quantity[pd.Series(w.products_types).isin(order.products_type)]:
                    routes.append(w) 
                else:
                    multi = []
                    multi.append(w)
                    quantity_left = np.array([order.products_quantity, w.products_quantity[pd.Series(w.products_types).isin(order.products_type)]]).min(0)
            
#             works till here
            
            if np.any(order.types.isin(w.types)):
                multi = [w]
                order_left = order[~order.types.isin(w.types)]
    #             for prod_type in order.types[~order.types.isin(w.types)]:
                for m in wharehouses:
                    if order_left.amounts < w[order_left.types == w.types].amounts                    
                        multi.append(m)
                    #check if the order is finished (amounts == 0
        return compute_best_routes(self, routes) #must compute the mininum over all routes but return the names of the wharehouses involved (the route)


In [25]:
type1order = 0
type2order = 0
for x in orders:
    type1order += (orders[x].tot_weight>200)
    type2order += (orders[x].tot_weight<=200)
type1order, type2order

(1028, 222)

In [11]:
class Warehouse:

    def __init__(self, num, x, y, amounts):
        quantity = np.array(amounts)
        weights = np.array(weight_prod_types)
        types = np.flatnonzero(quantity>0)
        self.num = num
        self.current_position = [x, y]
        self.products_types = types
        self.products_weights = weights[quantity >0]
        self.products_quantity = quantity[quantity >0]
        self.tot_weight = self.products_weights.sum()
        
    def table(self):
        return pd.DataFrame({'Weights': self.products_weights, 'Amounts': self.products_quantity}, index = self.products_types)
    
    def __repr__(self):
        return '(num: ' + str(self.num) + ', ' + 'n_products: '+ str(self.products_quantity.shape[0]) + ', ' + 'tot_weight: ' + str(self.tot_weight) + ', ' + 'tot_amount: ' + str(self.products_quantity.sum()) + ')'
    
    def add_product(self, prod_type, prod_qnt):
        self.product_type.append(prod_type)
        self.products_quantity.append(prod_qnt)

    def remove_product(self, prod_type, prod_qnt):
        self.products_quantity.remove(prod_qnt)
        if self.products_quantity == 0:
            self.product_type.remove(prod_type)


In [14]:
class Order:

    def __init__(self, num, x, y, amount, types):
        self.amount = amount
        weights = np.array(weight_prod_types)
        self.num = num
        self.position = [x, y]
        self.tot_weight = weights[types].sum()
        self.products_weights = weights[types]
        self.products_type = np.array(types)
        self.products_quantity = pd.Series(self.products_type).value_counts()
#         self.products_quantity = pd.Series(self.products_type, columns = 'type').groupby('type').count()
        self.completed = False
        self.turn_order_completed = 0
#         self.warehouses = #types and quantity 

    def table(self):
        return pd.DataFrame({'Weights': self.products_weights, 'Amounts': self.products_quantity}, index = self.products_type)
    #display only unique
    
    def __repr__(self):
        return '(num: ' + str(self.num) + ', ' + 'n_items: ' + str(self.products_type.shape[0]) + ', ' + 'tot_weight: ' + str(self.tot_weight) + ')'
    
    def add_product(self, prod_type, prod_qnt):
        self.product_type.append(prod_type)
        self.products_quantity.append(prod_qnt)

    def remove_product(self, prod_type, prod_qnt):
        self.product_type.remove(prod_type)
        self.products_quantity.remove(prod_qnt)
        
        
        

In [15]:
# initialize drones, warehouses and orders
drones = {}
for d in range(n_drones):
    drones[d] = Drone(d)
warehouses = {}
for w in range(n_wrhs):
    warehouses[w] = Warehouse(w, wrhs_info[w][0][0], wrhs_info[w][0][1], wrhs_info[w][1])
orders = {}
for o in range(n_orders):
    orders[o] = Order(o, order_info[o][0][0], order_info[o][0][1], order_info[o][1][0], order_info[o][2])
    
max_weight = max_payload #200 

for t in range(max_turns):
    
    for d in drones:
        nearest_order = d.find_nearest_order(orders)
        if nearest_order == 'All orders are completed'
            return "THE END"
        if (nearest_order.tot_weight <= max_weight):
            d.complete_order(nearest_order)
        
        
            